### importing dependaices

In [1]:
import pandas as pd
import requests
import json
import numpy as np
from pprint import pprint

### importing api key

In [2]:
from config import api_key

### reading in data from csv
#### filtered_fires.csv was filtered from origonal data in SQLiteStudio removing low/nominal confidence and non-vegetation fires

In [3]:
data = pd.read_csv (r'C:\Users\Ryan\Desktop\forest_fires\filtered_fires.csv')

### Snippet from readme file explaining the column values

https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/v1-vnp14imgt#ed-viirs-375m-attributes

Latitude: Center of nominal 375 m fire pixel

Longitude: Longitude	Center of nominal 375 m fire pixel

Bright_ti4: Brightness temperature I-4	VIIRS I-4 channel brightness temperature of the fire pixel measured in Kelvin.

Scan: Along Scan pixel size	The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.

Track: Along Track pixel size	The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.

Acq_Date: Acquisition Date	Date of VIIRS acquisition.
Acq_Time: Acquisition Time	Time of acquisition/overpass of the satellite (in UTC).
Satellite: Satellite	N= Suomi National Polar-orbiting Partnership (Suomi NPP)
Confidence: Confidence	
This value is based on a collection of intermediate algorithm quantities used in the detection process. It is intended to help users gauge the quality of individual hotspot/fire pixels. Confidence values are set to low, nominal and high. Low confidence daytime fire pixels are typically associated with areas of sun glint and lower relative temperature anomaly (<15K) in the mid-infrared channel I4. Nominal confidence pixels are those free of potential sun glint contamination during the day and marked by strong (>15K) temperature anomaly in either day or nighttime data. High confidence fire pixels are associated with day or nighttime saturated pixels.

Version: (Collection and source) Version identifies the collection (e.g. VIIRS Collection 1) and source of data processing: Near Real-Time (NRT suffix added to collection) or Standard Processing (collection only). "1.0NRT" - Collection 1 NRT processing. "1.0" - Collection 1 Standard processing.

Bright_ti5:	Brightness temperature I-5	I-5 Channel brightness temperature of the fire pixel measured in Kelvin.

FRP	(Fire Radiative Power): FRP depicts the pixel-integrated fire radiative power in MW (megawatts). Given the unique spatial and spectral resolution of the data, the VIIRS 375 m fire detection algorithm was customized and tuned in order to optimize its response over small fires while balancing the occurrence of false alarms. Frequent saturation of the mid-infrared I4 channel (3.55-3.93 µm) driving the detection of active fires requires additional tests and procedures to avoid pixel classification errors. As a result, sub-pixel fire characterization (e.g., fire radiative power [FRP] retrieval) is only viable across small and/or low-intensity fires. Systematic FRP retrievals are based on a hybrid approach combining 375 and 750 m data. In fact, starting in 2015 the algorithm incorporated additional VIIRS channel M13 (3.973-4.128 µm) 750 m data in both aggregated and unaggregated format.

Type:Inferred hot spot type	
    0 = presumed vegetation fire
    1 = active volcano
    2 = other static land source
    3 = offshore detection (includes all detections over water)
DayNight: Day or Night; D= Daytime fire, N= Nighttime fire

### creating the dataframe from csv file with header names

In [4]:
df = pd.DataFrame(data, columns = ['latitude',
                                   'longitude',
                                   'bright_ti4',
                                   'scan',
                                   'track',
                                   'acq_date',
                                   'acq_time',
                                   'satellite',
                                   'instrument',
                                   'confidence',
                                   'version',
                                   'bright_ti5',
                                   'frp',
                                   'daynight',
                                   'type'])
filtered_df = df.drop(['bright_ti4',
                       'scan',
                       'track',
                       'satellite',
                       'instrument',
                       'confidence',
                       'version',
                       'bright_ti5',
                       'daynight',
                       'type'], axis = 1)

In [5]:
filtered_df.head

<bound method NDFrame.head of         latitude   longitude    acq_date  acq_time    frp
0      19.418756 -155.083542  01/10/2015      1120   7.14
1      19.408396 -155.091934  01/10/2015      1120   8.82
2      19.413961 -155.091431  01/10/2015      1120  24.02
3      19.412861 -155.093277  01/10/2015      1120  22.31
4      19.421993 -155.060715  01/10/2015      1120  14.90
...          ...         ...         ...       ...    ...
59859  27.412804  -81.409821  11/11/2017      1752   9.22
59860  27.412285  -81.410515  11/11/2017      1752  10.40
59861  30.525211  -86.703514  11/11/2017      1753  12.33
59862  30.526947  -86.697243  11/11/2017      1753  12.33
59863  26.298752  -97.524200  11/11/2017      1933  11.39

[59864 rows x 5 columns]>

In [6]:
# filtered_df.to_csv("unrounded_data.csv", sep=',')

### rounding the lat and lon to simplify and group dataframe

In [7]:
rounded_df = filtered_df.round({'latitude':1, 'longitude':1})
rounded_df2 = rounded_df.rename(columns={"latitude": "round_lat",
                                         "longitude":"round_lon",
                                         "acq_date":"date",
                                         "acq_time":"time",
                                         "frp":"fire_pow"})

In [8]:
rounded_df2.head

<bound method NDFrame.head of        round_lat  round_lon        date  time  fire_pow
0           19.4     -155.1  01/10/2015  1120      7.14
1           19.4     -155.1  01/10/2015  1120      8.82
2           19.4     -155.1  01/10/2015  1120     24.02
3           19.4     -155.1  01/10/2015  1120     22.31
4           19.4     -155.1  01/10/2015  1120     14.90
...          ...        ...         ...   ...       ...
59859       27.4      -81.4  11/11/2017  1752      9.22
59860       27.4      -81.4  11/11/2017  1752     10.40
59861       30.5      -86.7  11/11/2017  1753     12.33
59862       30.5      -86.7  11/11/2017  1753     12.33
59863       26.3      -97.5  11/11/2017  1933     11.39

[59864 rows x 5 columns]>

In [9]:
result = pd.concat([rounded_df2, filtered_df], axis=1).reindex(rounded_df.index)
# result2 = result.drop_duplicates
# print(result2)
# # type(result2)
print(result)
result.info()

       round_lat  round_lon        date  time  fire_pow   latitude  \
0           19.4     -155.1  01/10/2015  1120      7.14  19.418756   
1           19.4     -155.1  01/10/2015  1120      8.82  19.408396   
2           19.4     -155.1  01/10/2015  1120     24.02  19.413961   
3           19.4     -155.1  01/10/2015  1120     22.31  19.412861   
4           19.4     -155.1  01/10/2015  1120     14.90  19.421993   
...          ...        ...         ...   ...       ...        ...   
59859       27.4      -81.4  11/11/2017  1752      9.22  27.412804   
59860       27.4      -81.4  11/11/2017  1752     10.40  27.412285   
59861       30.5      -86.7  11/11/2017  1753     12.33  30.525211   
59862       30.5      -86.7  11/11/2017  1753     12.33  30.526947   
59863       26.3      -97.5  11/11/2017  1933     11.39  26.298752   

        longitude    acq_date  acq_time    frp  
0     -155.083542  01/10/2015      1120   7.14  
1     -155.091934  01/10/2015      1120   8.82  
2     -155.0

In [10]:
result2 = result.drop(['acq_date', 'acq_time', 'fire_pow'],axis=1)
# modDfObj = dfObj.drop([dfObj.columns[1] , dfObj.columns[2]] ,  axis='columns')
# modDfObj = dfObj.drop(['Age' , 'Name'] , axis='columns')

### grouping by date/lat/lon and returning max FRP to get the highest FRP for unique day/location

In [11]:
df2 = result2.groupby(['round_lat', 'round_lon', 'date'])['frp'].transform(max) == result['frp']
print(df2)

0        False
1        False
2         True
3        False
4        False
         ...  
59859    False
59860     True
59861     True
59862     True
59863     True
Name: frp, Length: 59864, dtype: bool


In [12]:
df3 = result2[df2]
print(df3)

       round_lat  round_lon        date  time   latitude   longitude     frp
2           19.4     -155.1  01/10/2015  1120  19.413961 -155.091431   24.02
11          32.0      -91.7  01/10/2015  1911  31.958311  -91.721252   81.70
12          30.8      -95.0  01/10/2015  1911  30.798439  -95.018036   69.37
13          31.1      -90.8  01/10/2015  1911  31.059824  -90.767311  128.65
14          31.1      -90.8  01/10/2015  1911  31.056522  -90.766647  128.65
...          ...        ...         ...   ...        ...         ...     ...
59858       26.5      -80.5  11/11/2017  1752  26.513035  -80.520752   14.68
59860       27.4      -81.4  11/11/2017  1752  27.412285  -81.410515   10.40
59861       30.5      -86.7  11/11/2017  1753  30.525211  -86.703514   12.33
59862       30.5      -86.7  11/11/2017  1753  30.526947  -86.697243   12.33
59863       26.3      -97.5  11/11/2017  1933  26.298752  -97.524200   11.39

[23637 rows x 7 columns]


### condensed forest fire dataframe

#### consists of single FRP value for each simplified location
#### need to left-join on index long-form lat/long from 'filtered_df' to do marker placement
#### final DF should also have formatted location data (reverse google-API call) for marker labels

### transforming dataframe to dictionary to execute for-loops
#### perform API calls - formatted address, marker placement

In [13]:


fire_list = [df3.columns.values.tolist()] + df3.values.tolist()

# fire_list.to_csv("fire_list.csv", sep=',')
fire_list_no_header = fire_list[1:]
header = fire_list[:1]
print(header)
# print(fire_list_no_header)

[['round_lat', 'round_lon', 'date', 'time', 'latitude', 'longitude', 'frp']]


In [14]:
fire_df = pd.DataFrame(fire_list_no_header)
fire_df.columns = ['round_lat', 'round_lon', 'date', 'time', 'latitude', 'longitude', 'frp']
fire_df.to_csv("fire_df.csv", sep=',')

In [15]:
### reverse geodata api call to get location based on lat/lon

lat = fire_list_no_header[0][4]
lon = fire_list_no_header[0][5]
target_url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
geo_data = requests.get(target_url).json()
pprint(geo_data)
formatted_address = (geo_data["results"][1]["formatted_address"])
# print(lat)

{'plus_code': {'compound_code': 'CW75+HC Fern Forest, HI, USA',
               'global_code': '73F6CW75+HC'},
 'results': [{'address_components': [{'long_name': '96778',
                                      'short_name': '96778',
                                      'types': ['postal_code']},
                                     {'long_name': 'Pāhoa',
                                      'short_name': 'Pāhoa',
                                      'types': ['locality', 'political']},
                                     {'long_name': 'Hawaii County',
                                      'short_name': 'Hawaii County',
                                      'types': ['administrative_area_level_2',
                                                'political']},
                                     {'long_name': 'Hawaii',
                                      'short_name': 'HI',
                                      'types': ['administrative_area_level_1',
                               

In [16]:
print(formatted_address)

Island of Hawai'i, Hawaii, USA


In [17]:
fire_loc = []

for i in fire_list_no_header:
    lat = i[4]
    lon = i[5]
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
    geo_data = requests.get(target_url).json()
    formatted_address = (geo_data["results"][1]["formatted_address"])
    print(formatted_address)
    fire_loc.append(formatted_address)
#     pprint(geo_data)
#     frp = i[6]
#     print(f'[{lat},{lon}]')

Island of Hawai'i, Hawaii, USA
670 Berry Ln, Wisner, LA 71378, USA
Segno, TX 77351, USA
Liberty, MS 39645, USA
4839 Mary Wall Bridge Rd, Liberty, MS 39645, USA
8118-7968 Co Rd 28, Mountain View, AR 72560, USA
Monroe Township, AR, USA
27934-28420 Weeks Rd, Waldron, AR 72958, USA
Western Grove, AR 72685, USA
Arkansas Township, AR, USA
De Bastrop Township, AR, USA
De Bastrop Township, AR, USA
6600-8004 Cornerstone Rd, Altheimer, AR 72004, USA
29666 AR-75, Marked Tree, AR 72365, USA
4455-4553 Island 40 Rd, Marion, AR 72364, USA
3118-3134 Wilson Rd, Wabbaseka, AR 72175, USA
3829 Bill Mac Rd, Carlisle, AR 72024, USA
841 G Rd, Larned, KS 67550, USA
Truchas, NM 87578, USA
16139-17499 E Trimmer Springs Rd, Sanger, CA 93657, USA
Island of Hawai'i, Hawaii, USA
Island of Hawai'i, Hawaii, USA
8061-8001 FM457, Bay City, TX 77414, USA
Matagorda County, TX, USA
Matagorda County, TX, USA
Wiergate, TX 75977, USA
1419 Co Rd 212, Falfurrias, TX 78355, USA
1900 AR-277, Watson, AR 71674, USA
San Angelo, TX 

KeyboardInterrupt: 

In [ ]:
print(fire_loc[0:5])